Cesar Notebook, do test stuff here to aid in keeping main clean

Basic Imports/ Notebook setup
it brokey

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch import nn, optim

#setting our device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the dataset for the training/test sets
def get_data(batch_size: int) -> Tuple[DataLoader, DataLoader]:
    
    train = torchvision.datasets.FashionMNIST(root="./",train=True,transform= ToTensor())

    test_dataset = torchvision.datasets.FashionMNIST(root="./", transform = ToTensor(), train=False)    

    train_DL = DataLoader(train, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    return (train_DL, test_DL)


class NeuralNetwork(nn.Module):


    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
      
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out
    


In [94]:
def subset_data(batch_size : int,subset : list) -> Tuple[DataLoader, DataLoader]:
    test_dataset = torchvision.datasets.FashionMNIST(root="./", transform = ToTensor(), train=False)    
    train_sub = DataLoader(subset, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    return (train_sub, test_DL)

In [4]:
#making our model

model = NeuralNetwork()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
data_set = get_data(200)


In [11]:

num_epochs = 5
count = 0
# Lists for visualization of loss and accuracy 
loss_list = []
iteration_list = []
accuracy_list = []

# Lists for knowing classwise accuracy
predictions_list = []
labels_list = []
for epoch in range(num_epochs):
    for images, labels in data_set[0]:
        # Transfering images and labels to GPU if available
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(200, 1, 28, 28))
        labels = Variable(labels)
        # Forward pass 
        outputs = model(train)
        loss = error(outputs, labels)
        
        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()
        
        #Propagating the error backward
        loss.backward()
        
        # Optimizing the parameters
        optimizer.step()
    
        count += 1
    
    # Testing the model
    
        if not (count % 50):    # It's same as "if count % 50 == 0"
            total = 0
            correct = 0
        
            for images, labels in data_set[1]:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(200, 1, 28, 28))
            
                outputs = model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
                   

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

KeyboardInterrupt: 

Data "Distilling" by that I mean subsetting

make data into list bc lol pytorch dataset class is BAD

In [93]:
#import random numpy library for subset creation
from numpy import random as rd

#load trainset data
train_set_sub = torchvision.datasets.FashionMNIST("./")

#convert to list for easy data deletion
train_set_l = list(train_set_sub)

<class 'torchvision.datasets.mnist.FashionMNIST'>
<class 'tuple'>


In [1]:
#subset using a random uniform dist
def uniform(train_set_l : list):
    while len(train_set_l) > 30000:
        randomVal = rd.uniform(0,1)
        a = 0
        b = len(train_set_l)
        temp = randomVal
        temp2 = np.floor((a + (b-a)) * temp)
        del train_set_l[temp2]
    return train_set_l

#subset using a random accept reject
def AcRj(train_set_l: list):
    while len(train_set_l) > 30000:
        del_int = rd.randint(0,60000)
        if del_int > len(train_set_l) - 1:
            while del_int > len(train_set_l) - 1:
                del_int = rd.randint(0,60000)
            del train_set_l[del_int]
        else:
            del train_set_l[del_int]
    return train_set_l

#subset using deleting with range
def Ac(train_set_l: list):
    while len(train_set_l) > 30000:
        del_int = rd.randint(0,len(train_set_l))
        del train_set_l[del_int]
    return train_set_l

NameError: name 'train_set_l' is not defined

In [92]:
training_l = 
#make data loaders of new subsetted data
data_sub = subset_data(200, train_set_l)